# 행정동 법정동 변환

In [ ]:
# === 0) 드라이브 마운트 ===
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 단기체류외국인

In [ ]:
|import pandas as pd
import re

# 1) 엑셀 읽기: 헤더 1줄
df_raw = pd.read_excel(
    "/content/drive/MyDrive/2025 빅콘/사용할 데이터/외부인 영향 지표/단기체류외국인.xlsx",
    header=0
)

# 2) 컬럼 이름 통일 (필요 시)
df_raw = df_raw.rename(columns={"행정동": "행정동"})

# 3) 행정동 정규화
def norm_admin(name: str) -> str:
    s = str(name)
    s = re.sub(r"\s+", "", s)                 # 모든 공백 제거
    s = re.sub(r"제(\d+)동", r"\1동", s)       # 왕십리제2동 -> 왕십리2동, 행당제1동 -> 행당1동
    # 금호 2,3가동 표기들 통일
    s = s.replace("금호 2,3가동", "금호2·3가동").replace("금호2,3가동", "금호2·3가동")
    s = s.replace("금호2.3가동", "금호2·3가동").replace("금호2ㆍ3가동", "금호2·3가동")
    # 성수1가/2가 제n동 표기 통일
    s = s.replace("성수1가제1동", "성수1가1동").replace("성수1가제2동", "성수1가2동")
    s = s.replace("성수2가제1동", "성수2가1동").replace("성수2가제3동", "성수2가3동")
    # 행당
    s = s.replace("행당제1동", "행당1동").replace("행당제2동", "행당2동")
    # 왕십리
    s = s.replace("왕십리제2동", "왕십리2동")
    return s

df = df_raw.copy()
df["행정동"] = df["행정동"].apply(norm_admin)

# 4) 분기/월 컬럼만 선택 (행정동코드 제외)
quarter_cols = [c for c in df.columns if c not in ["행정동", "행정동코드"]]

# 5) 인덱스 설정
df = df.set_index("행정동")

# 6) 안전 조회 함수
def row_val(admin_name: str):
    key = norm_admin(admin_name)
    if key not in df.index:
        # 근처 후보를 보여주면 디버깅이 쉬움
        nearby = [idx for idx in df.index if any(tok in idx for tok in key[:3])]
        raise KeyError(f"[행정동 없음] '{admin_name}' -> '{key}' 가 데이터에 없습니다. "
                       f"비슷한 후보: {nearby[:10]}")
    return pd.to_numeric(df.loc[key, quarter_cols], errors="coerce")


In [ ]:
legal_index = [
    "상왕십리동","홍익동","도선동","하왕십리동","마장동","사근동","행당동","응봉동",
    "금호동1가","금호동2가","금호동3가","금호동4가","옥수동",
    "성수동1가","성수동2가","송정동","용답동"
]
result = pd.DataFrame(index=legal_index, columns=quarter_cols, dtype=float)

simple_map = {
    "상왕십리동": "왕십리도선동",
    "홍익동": "왕십리도선동",
    "도선동": "왕십리도선동",
    "마장동": "마장동",
    "사근동": "사근동",
    "응봉동": "응봉동",
    "금호동1가": "금호1가동",
    "금호동2가": "금호2·3가동",   # <- 통일!
    "금호동3가": "금호2·3가동",   # <- 통일!
    "금호동4가": "금호4가동",
    "옥수동": "옥수동",
    "송정동": "송정동",
    "용답동": "용답동",
}
for law, admin in simple_map.items():
    result.loc[law] = row_val(admin).values

# 평균 매핑
result.loc["하왕십리동"] = (row_val("왕십리도선동") + row_val("왕십리2동")) / 2
result.loc["행당동"]     = (row_val("사근동") + row_val("행당1동") + row_val("행당2동")) / 3
result.loc["성수동1가"]  = (row_val("성수1가1동") + row_val("성수1가2동")) / 2
result.loc["성수동2가"]  = (row_val("성수2가1동") + row_val("성수2가3동")) / 2

result = result.sort_index()
result.to_excel("단기체류외국인_법정동.xlsx")
print("완료: 단기체류외국인_법정동.xlsx")


완료: 단기체류외국인_법정동.xlsx


## 지역별 내국인 방문자 수 비율

In [ ]:
import pandas as pd
import re
from pathlib import Path

# ── 1) 행정동 표기 정규화 ─────────────────────────────────────────────
def norm_admin(name: str) -> str:
    s = str(name)
    s = re.sub(r"\s+", "", s)                  # 모든 공백 제거
    s = re.sub(r"제(\d+)동", r"\1동", s)         # 왕십리제2동 → 왕십리2동, 행당제1동 → 행당1동
    # 금호 2,3가동 표기 통일 (., , ㆍ, · 모두 수용 → '금호2·3가동'으로 통일)
    s = (s.replace("금호 2,3가동","금호2·3가동")
           .replace("금호2,3가동","금호2·3가동")
           .replace("금호2.3가동","금호2·3가동")
           .replace("금호2ㆍ3가동","금호2·3가동"))
    # 성수1가/2가 제n동 표기 통일
    s = (s.replace("성수1가제1동","성수1가1동")
           .replace("성수1가제2동","성수1가2동")
           .replace("성수2가제1동","성수2가1동")
           .replace("성수2가제3동","성수2가3동"))
    # 행당
    s = s.replace("행당제1동","행당1동").replace("행당제2동","행당2동")
    # 왕십리
    s = s.replace("왕십리제2동","왕십리2동")
    return s

# ── 2) 월/분기 컬럼명 표준화: 2023.1 → 2023.01 ────────────────────────
def norm_period(col: str) -> str:
    c = str(col).strip()
    m = re.match(r"^(\d{4})\.(\d{1,2})$", c)
    if m:
        return f"{m.group(1)}.{int(m.group(2)):02d}"
    # 혹시 '2023.01 ' 같은 경우도 커버
    m = re.match(r"^(\d{4})\.(\d{1,2})\D*$", c)
    if m:
        return f"{m.group(1)}.{int(m.group(2)):02d}"
    return c

# ── 3) 파일 → 법정동 매핑 실행 함수 ──────────────────────────────────
def build_legal_table(path_in, path_out):
    # 1) 읽기
    df_raw = pd.read_excel(path_in, header=0)

    # 첫 컬럼명을 '행정동'으로 강제
    first_col = df_raw.columns[0]
    if str(first_col) != "행정동":
        df_raw = df_raw.rename(columns={first_col: "행정동"})

    # 2) 컬럼명(월/분기) 표준화
    df_raw = df_raw.rename(columns={c: norm_period(c) for c in df_raw.columns})

    # >>> [추가] 표준화 이후 중복 컬럼 정리 (왼쪽 첫 컬럼을 남김) <<<
    if df_raw.columns.duplicated().any():
        # 1) 가장 간단: 첫 컬럼만 유지
        df_raw = df_raw.loc[:, ~df_raw.columns.duplicated(keep='first')]
        # 만약 중복열을 평균/최신값으로 합치고 싶다면 아래 대안 사용
        # df_raw = df_raw.T.groupby(level=0).first().T    # 첫번째 비결측
        # df_raw = df_raw.T.groupby(level=0).mean(numeric_only=True).T  # 평균

    # 3) 정규화 & 인덱스/수치 변환
    df = df_raw.copy()
    df["행정동"] = df["행정동"].apply(norm_admin)

    value_cols = [c for c in df.columns if c not in ["행정동","행정동코드"]]

    # >>> [수정] 개별 컬럼 루프 대신 DataFrame 단위로 숫자 변환 <<<
    df[value_cols] = (df[value_cols]
                      .replace({',': ''}, regex=True)   # 쉼표 제거(있을 경우)
                      .replace({'%': ''}, regex=True)   # % 제거(있을 경우)
                      .apply(pd.to_numeric, errors='coerce'))

    df = df.set_index("행정동")

    # 4) 안전 조회 함수 (동일)
    def row_val(admin_name: str):
        key = norm_admin(admin_name)
        if key not in df.index:
            hints = [idx for idx in df.index if any(tok in idx for tok in [key[:2], key[-2:]])]
            raise KeyError(f"[행정동 없음] '{admin_name}' → '{key}' 가 데이터에 없습니다. 후보: {hints[:10]}")
        return df.loc[key, value_cols]

    # 5) 결과 테이블 구성(동일)
    legal_index = [
        "상왕십리동","홍익동","도선동","하왕십리동","마장동","사근동","행당동","응봉동",
        "금호동1가","금호동2가","금호동3가","금호동4가","옥수동",
        "성수동1가","성수동2가","송정동","용답동"
    ]
    result = pd.DataFrame(index=legal_index, columns=value_cols, dtype=float)

    simple_map = {
        "상왕십리동": "왕십리도선동",
        "홍익동": "왕십리도선동",
        "도선동": "왕십리도선동",
        "마장동": "마장동",
        "사근동": "사근동",
        "응봉동": "응봉동",
        "금호동1가": "금호1가동",
        "금호동2가": "금호2·3가동",
        "금호동3가": "금호2·3가동",
        "금호동4가": "금호4가동",
        "옥수동": "옥수동",
        "송정동": "송정동",
        "용답동": "용답동",
    }
    for law, admin in simple_map.items():
        result.loc[law] = row_val(admin).values

    result.loc["하왕십리동"] = (row_val("왕십리도선동") + row_val("왕십리2동")) / 2
    result.loc["행당동"]     = (row_val("사근동") + row_val("행당1동") + row_val("행당2동")) / 3
    result.loc["성수동1가"]  = (row_val("성수1가1동") + row_val("성수1가2동")) / 2
    result.loc["성수동2가"]  = (row_val("성수2가1동") + row_val("성수2가3동")) / 2

    result = result.sort_index()
    from pathlib import Path
    Path(path_out).parent.mkdir(parents=True, exist_ok=True)
    result.to_excel(path_out)
    print(f"완료: {path_out}")


# ── 실행 예시 ─────────────────────────────────────────────────────────
build_legal_table("/content/drive/MyDrive/2025 빅콘/사용할 데이터/외부인 영향 지표/지역별 외국인 지출액 비율.xlsx",
                   "지역별 외국인 지출액 비율_법정동.xlsx")


완료: 지역별 외국인 지출액 비율_법정동.xlsx
